In [1]:
#import libraries

import pandas as pd
from pandas.tseries.offsets import *
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import math

from bokeh.layouts import column, row
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select, LinearAxis, Range1d
from bokeh.io import output_notebook
from bokeh.models.widgets import DataTable, TableColumn, CheckboxGroup, Paragraph, Button

In [2]:
#setup

stocks = pd.Series({'CSCO' :'Cisco',
                    'IBM'  :'IBM',
                    'INTC' :'Intel',
                    'MSFT' : 'Microsoft',
                    'VZ'   : 'Verizon'})
tickers= list(stocks.index)       

macro = {'PMI':['ISM/MAN_PMI','ISM Composite PMI Index (PMI)'],
         'CCI':['UMICH/SOC1','Consumer Confidence Index (CCI)'],
         'RTSL':['FRED/RETAILMPCSMSA','Retail Sales (yoy) (RTSL)'],
         'UST10':['FRED/DGS10','US 10-year Treasury Yield (UST10)'],
         'NFIC':['FRED/NFCI','Financial Conditions Index (NFCI)']}
macro_var = list(macro.keys())

start = '2013-01-01' #free EPS data from quandl goes back as far as 2013

COE = 0.1            #cost of equity (discount rate) assumption


In [3]:
#define dates
start = pd.Timestamp(start).date()
end   = pd.Timestamp(pd.Timestamp.today().year + 1, 12, 31).date() #end of next year
today = pd.Timestamp(pd.Timestamp.today()).date() - BDay(1)
cutoff = start + pd.DateOffset(months=3)

In [4]:
import quandl
quandl.ApiConfig.api_key = 'YOUR_API_KEY' #insert your own quandl API key

#quandl resources:
#https://docs.quandl.com/docs/python-installation
#https://docs.quandl.com/docs/python-time-series
#https://github.com/quandl/quandl-python/blob/master/FOR_DEVELOPERS.md

In [5]:
#create historical EPS series for fiscal year FY0 (current), FY1 (next), FY2, FY3
def Create_EPS_Series(df, fy):
    #creates an EPS series for a given fiscal year (FY) from a historical EPS est table from Quandl (ZACKS/EEH)
    eps = df.copy()
    eps = eps[eps.obs_date <= eps.per_end_date]
    eps.loc[eps.obs_date.dt.year + fy == eps.per_end_date.dt.year, 'eps'] = 'True'
    eps.dropna(inplace=True)
    eps.set_index('obs_date', inplace=True)
    eps.loc[fy0,'eps_mean_est'] = eps.loc[eps.index.max(), 'eps_mean_est']
    eps.drop(['ticker','per_end_date','eps'], axis=1, inplace=True)
    eps.columns = ['EPS' + str(fy)]
    df = pd.DataFrame(pd.date_range(start, fy0, freq='D'), columns=['obs_date']).set_index('obs_date')
    df = df.merge(eps, how='left', left_index=True, right_index=True)
    df.index.names = ['Date']
    df.fillna(method='ffill', inplace=True)  
    return (df)

#import data
df_e = pd.DataFrame(pd.date_range(start, end, freq='D'), columns=['Date']).set_index('Date') 
for ticker in range(len(tickers)):  
    columns=['ticker','obs_date','per_end_date','eps_mean_est']
    df_q = quandl.get_table('ZACKS/EEH', ticker=tickers[ticker],
                            per_type='A', qopts={'columns': columns},
                            paginate=True)

    #define fiscal year for timeseries
    for y in range (3):
        vars()['fy'+str(y)] = pd.Timestamp(pd.Timestamp.today().year + y,
                                           df_q.loc[0, 'per_end_date'].month,
                                           df_q.loc[0, 'per_end_date'].day)
    #print (tickers[ticker], ' | fy0 =', fy0.date(), '| fy1 =', fy1.date(), '| fy2 =', fy2.date())
    
    #create forward looking EPS series for fiscal year FY0 (current), FY1 (next), FY2
    #methodology: carrying forward values. After FY2, move to terminal growth (LTG)
    df = pd.DataFrame(pd.date_range(start, end, freq='D'), columns=['Date']).set_index('Date') 
    for fy in range(3):  
        df = df.merge(Create_EPS_Series(df_q, fy), how='left', left_index=True, right_index=True)

    #EGR2 = EPS growth in year 2
    EGR2 = df.loc[fy0, 'EPS2'] / df.loc[fy0, 'EPS1'] - 1
    
    #EPS0 roll at the end of the fiscal year
    df.loc[fy1, 'EPS0'] = df.loc[fy0, 'EPS1']
    df.loc[fy2, 'EPS0'] = df.loc[fy0, 'EPS2']

    #EPS1 roll at the end of the fiscal year
    df.loc[fy1, 'EPS1'] = df.loc[fy0, 'EPS2']
    df.loc[fy2, 'EPS1'] = df.loc[fy0, 'EPS2'] * (1+EGR2)

    #EPS2 roll at the end of the fiscal year
    df.loc[fy1, 'EPS2'] = df.loc[fy0, 'EPS2'] * (1+EGR2)
    df.loc[fy2, 'EPS2'] = df.loc[fy0, 'EPS2'] * (1+EGR2)**2
    
    df[today:] = df[today:].fillna(method='bfill')
    
    #create day fraction
    fye = pd.DataFrame(pd.Series(df_q.per_end_date.unique()), columns=['Date']).set_index('Date')    
    fye['FYE'] = fye.index
    df = df.merge(fye, how='left', left_index=True, right_index=True)
    df.FYE.fillna(method='bfill', inplace=True)
    df['DayFrac'] = (df.FYE - df.index).dt.days / 365

    #create blended forward series (EPS_BF1, EPS_BF2), and EPS growth series
    df['EPS_BF1'] = df.DayFrac * df.EPS0 + (1-df.DayFrac) * df.EPS1
    df['EPS_BF2'] = df.DayFrac * df.EPS1 + (1-df.DayFrac) * df.EPS2
    df['G_BF2'] = (df.EPS2 / df.EPS1)-1
    df['G_CAGR2'] = ((df.EPS2 / df.EPS0) ** 0.5) -1
    
    #remove working columns and add ticker
    df.drop(['FYE','DayFrac'], axis=1, inplace=True)
    df.columns = [tickers[ticker]+'_EPS0',
                  tickers[ticker]+'_EPS1',
                  tickers[ticker]+'_EPS2',
                  tickers[ticker]+'_EPS_BF1',
                  tickers[ticker]+'_EPS_BF2',
                  tickers[ticker]+'_G_BF2',
                  tickers[ticker]+'_G_CAGR2']
    
    #add stock specific data to df_e
    df_e = df_e.merge(df, how='left', left_index=True, right_index=True) 

In [6]:
#import prices
df_px = pd.DataFrame(pd.date_range(start, end, freq='D'), columns=['Date']).set_index('Date')    
for i in range(len(tickers)):
    temp = quandl.get('EOD/'+tickers[i], start_date=start)
    temp = temp[['Close']]
    temp.columns = [tickers[i]+'_PX']
    df_px = df_px.merge(temp, how='left', left_index=True, right_index=True)
df_px.fillna(method='ffill', inplace=True)

#create return series
for ticker in range(len(tickers)):
    df_px[tickers[ticker] + '_RT'] = df_px[tickers[ticker] + '_PX'].pct_change(periods=1)
df_px[today:] = np.nan

In [7]:
#import macro data
df_m = pd.DataFrame(pd.date_range(start, end, freq='D'), columns=['Date']).set_index('Date')    
for var in macro:
    temp = quandl.get(macro[var][0], start_date=start)
    temp.columns = [var]
    df_m = df_m.merge(temp, how='left', left_index=True, right_index=True)
df_m.fillna(method='ffill', inplace=True)

In [8]:
#merge earnings, price and macro data into one dataframe, and create price earnings ratios
df = pd.DataFrame(pd.date_range(start, end, freq='D'), columns=['Date']).set_index('Date') 
df = df.merge(df_px, how='left', left_index=True, right_index=True)
df = df.merge(df_e, how='left', left_index=True, right_index=True)
df = df.merge(df_m, how='left', left_index=True, right_index=True)

#create price earnings ratios
for ticker in range(len(tickers)):
    df[tickers[ticker] + '_PE'] = df[tickers[ticker] + '_PX'] / df[tickers[ticker] + '_EPS_BF1']

#calc implied LTG: LTG = COE - k / PE; we assume k=1
for i in range(len(tickers)):
    df[tickers[i]+'_LTG'] = COE - 1/df[tickers[i]+'_PE']
    
#cut off first 3 months with NA values and resample, business days only
df = df[cutoff:]
df = df.resample('B').last()

#check for missing values
if df[:today].isnull().any(axis=1).sum() != 0:
    print ('Dataframe check: there are missing values')
else:
    print ('Dataframe check: ok, no missing values')

#df.to_excel("df.xlsx", sheet_name='Sheet1')   #export df into excel, if needed

Dataframe check: there are missing values


In [9]:
#calc and display correlation with macro variables
df1 = df.copy()
df1.dropna(inplace=True)
A = np.zeros(shape=(len(tickers),len(macro_var)))
for i in range(len(tickers)):
    for j in range(len(macro_var)):
        A[i,j] = np.corrcoef(df1[tickers[i] + '_LTG'].values, df1[macro_var[j]].values)[1,0]
A = pd.DataFrame(A, index=tickers, columns=macro_var)
A.round(2).style.background_gradient(cmap='coolwarm')

,PMI,CCI,RTSL,UST10,NFIC
CSCO,0.57,0.73,-0,0.41,-0.53
IBM,0.15,0.06,0.19,-0.19,-0.01
INTC,-0.16,-0.19,-0,-0.24,0.06
MSFT,0.36,0.8,0.05,0.14,-0.35
VZ,-0.22,-0.66,0.01,-0.26,0.27


In [10]:
#ARX(1) model

import warnings
warnings.filterwarnings("ignore")

def ARX(endo, exog):

    endo1 = endo.values    
    exog1 = exog.values
      
    try:      
        #fit model
        model = sm.tsa.statespace.SARIMAX(endo1, exog=exog1,order=(1,0,0), trend='c', seasonal_order=(0,0,0,0))
        results=model.fit()
        #print (results.summary())
        
        #run predictions
        pred = results.predict(start=0, end=None, exog=exog1, dynamic=True)
        pred = pd.Series(pred, index=pd.date_range(cutoff, end, freq='B'))
    
        #get predicted mean
        predict   = results.get_prediction()
        pred_mean = predict.predicted_mean
        pred_mean = pd.Series(pred_mean, index=pd.date_range(cutoff, end, freq='B'))
        
        #get accuracy metrics: ADF, p-values, RMSE
        pvalues = results.pvalues               #p-values of indep var, <0.05 = statistically significant
        pvalues = pvalues[1:][:-1].round(4)
        resid = endo[:today] - pred[:today]
        RMSE = np.sqrt(np.sum(resid**2)/len(resid)).round(4)            #Root mean squared error (RMSE)
        accur = np.append(pvalues, RMSE)
        #ADF = adfuller(endo[:today].values, autolag='AIC')[1].round(4)  #dickey-fuller test for stationarity
        #accur = np.append(accur, ADF)
        
        #plot acf, pacf plots (diagnosis only, comment out for production)
        #plot_acf(exog1, lags=25)
        #plot_pacf(exog1, lags=25)
        #plt.show()
           
    except:
        #return an empty array if the regression fails
        pred = pd.Series(np.nan, index=pd.date_range(cutoff, end, freq='B'))
        pred_mean = pd.Series(np.nan, index=pd.date_range(cutoff, end, freq='B'))
        accur = []

    return (pred, pred_mean, accur)

In [11]:
#create bokeh chart
output_notebook()

#checkbox options
options = {}
for i in macro:
    options[i] = macro[i][1]
#options['CAGR2'] = 'EPS 2-Year CAGR' #additional variables can be added

#starting values for charts and table
initial = tickers[0]
dt = pd.Series(index=pd.date_range(cutoff, end, freq='B')).index
empty = pd.Series(np.nan, index=pd.date_range(cutoff, end, freq='B'))
PX = df.loc[:, initial+'_PX']
EPS = df.loc[:, initial+'_EPS_BF1']
LTG = df.loc[:, initial+'_LTG']
PE = df.loc[:, initial+'_PE']
PredLTG = empty
FairPE = empty
FairPX = empty
stats = dict(var=[],val=[])

def bokeh_plot(doc):

    #update datasets
    sourcePX = ColumnDataSource(data={'x': dt, 'y': PX})
    sourceEPS = ColumnDataSource(data={'x': dt, 'y': EPS})
    sourceLTG = ColumnDataSource(data={'x': dt, 'y': LTG})
    sourcePE = ColumnDataSource(data={'x': dt, 'y': PE})
    sourcePredLTG = ColumnDataSource(data={'x': dt, 'y': PredLTG})
    sourceFairPE = ColumnDataSource(data={'x': dt, 'y': FairPE})
    sourceTable = ColumnDataSource(data=stats)
    sourceFairPX = ColumnDataSource(data={'x': dt, 'y': FairPX})
    
    #specify plot0: EPS
    plot0 = figure(x_axis_label='Date', y_axis_label='Share Price ($)',
                   title='Share Price vs Earnings', plot_width=425, plot_height=250, x_axis_type='datetime')
    plot0.line(x= 'x', y='y', legend='Price', source=sourcePX, line_width=1, color='black')
    plot0.legend.location = "top_left"
    plot0.title.text_font_size = "16px"  
    
    #specify plot0, 2nd y-axis (requires a specified min/max value)
    Emin = math.floor(EPS.min()*0.9)
    Emax = math.ceil(EPS.max()*1.1)
    plot0.extra_y_ranges = {'EPS': Range1d(start=Emin, end=Emax)}
    plot0.add_layout(LinearAxis(y_range_name='EPS', axis_label='Earnings Per Share ($)'), 'right')
    plot0.line(x= 'x', y='y', legend='EPS', source=sourceEPS, y_range_name="EPS", line_width=1, color='green')
    
    #specify plot1: LTG
    plot1 = figure(x_axis_label='Date', y_axis_label='Long-Term Growth (%)',
                   title='Long Term Growth', plot_width=375, plot_height=250, x_axis_type='datetime')
    plot1.line(x= 'x', y='y', legend='Market-implied LTG', source=sourceLTG, line_width=1, color='black')
    plot1.line(x= 'x', y='y', legend='Predicted LTG', source=sourcePredLTG, line_width=2, color='orange')
    plot1.legend.location = "top_left"
    plot1.title.text_font_size = "16px"  
    
    #specify plot2: FairPE
    plot2 = figure(x_axis_label='Date', y_axis_label='Price-to-Earnings Ratio',
                   title='Valuation Model: P/E Ratio', plot_width=425, plot_height=250, x_axis_type='datetime')
    plot2.line(x= 'x', y='y', legend='Actual P/E', source=sourcePE, line_width=1, color='black')
    plot2.line(x= 'x', y='y', legend='Predicted P/E', source=sourceFairPE, line_width=2, color='red')
    plot2.legend.location = "top_left"
    plot2.title.text_font_size = "16px"
    
    #specify plot3: FairPX
    plot3 = figure(x_axis_label='Date', y_axis_label='Share Price ($)',
                   title='Actual vs Predicted Share Price', plot_width=375, plot_height=250,
                   x_axis_type='datetime')
    plot3.line(x= 'x', y='y', legend='Actual Price', source=sourcePX, line_width=1, color='black')
    plot3.line(x= 'x', y='y', legend='Predicted Price', source=sourceFairPX, line_width=2, color='blue')
    plot3.legend.location = "top_left"
    plot3.title.text_font_size = "16px"    
    
    #specify table with accuracy metrics
    col = [TableColumn(field="var", title="Metric"), TableColumn(field="val", title="Value")]
    table = DataTable(source=sourceTable, columns=col, width=175, height=250)
    
    #specify widgets
    #adding widgets https://bokeh.pydata.org/en/latest/docs/user_guide/interaction/widgets.html
    menu = Select(title='Ticker', value=tickers[0], options=tickers)
    check = CheckboxGroup(labels=list(options.values()))
    refresh = Button(label="Refresh", button_type="primary", width=140)
    textbox1 = Paragraph(text='', width=370, height=10)
    textbox2 = Paragraph(text='', width=230, height=100)
    textbox3 = Paragraph(text='', width=100, height=50)
    
    #function that is triggered when user changes any of the values
    ###def callback(attr, old, new):
    def callback():
        
        #update sourcePX
        PX = df.loc[:, menu.value+'_PX']
        sourcePX.data={'x': dt, 'y': PX}
        
        #update sourceEPS
        EPS = df.loc[:, menu.value+'_EPS_BF1']
        sourceEPS.data={'x': dt, 'y': EPS}
        #update y-axis of EPS chart
        Emax = math.ceil(EPS.max()*1.1)
        Emin = math.floor(EPS.min()*0.9)
        plot0.extra_y_ranges['EPS'].start = Emin
        plot0.extra_y_ranges['EPS'].end = Emax
    
        #update sourceLTG
        LTG = df.loc[:, menu.value+'_LTG']
        sourceLTG.data={'x': dt, 'y': LTG}  
        
        #update sourcePE
        PE = df.loc[:, menu.value+'_PE']
        sourcePE.data={'x': dt, 'y': PE}  
        
        #update sourcePredLTG, sourceFairPE, sourceFairPX if regression specified ie variables selected
        exog_var = []
        for i in check.active:
            exog_var.append(list(options.keys())[i])
   
        if exog_var == []:
            sourcePredLTG.data={'x': dt, 'y': empty}
            sourceFairPE.data={'x': dt, 'y': empty}
            sourceFairPX.data={'x': dt, 'y': empty}
            sourceTable.data = dict(var=[],val=[]) 
            
        else:
            depvar=df.loc[:, menu.value+'_LTG'].copy()
            #if exog_var[-1] == 'CAGR2':    #to incl additional variables
            #    exog_var[-1] = menu.value+'_G_CAGR2'
            indepvar=df.loc[:, exog_var].copy()
            pred, pred_mean, accur = ARX(depvar, indepvar)
            
            try:
                PredLTG=pred
                sourcePredLTG.data={'x': dt, 'y': PredLTG}
                
                FairPE =1/(COE-pred)
                sourceFairPE.data={'x': dt, 'y': FairPE}

                FairPX = FairPE * EPS
                sourceFairPX.data={'x': dt, 'y': FairPX}
                
                metrics = []
                for i in range(len(exog_var)):
                    metrics.append(exog_var[i]+' p-val')
                metrics.extend(['AR(1) p-val','RMSE (LTG)'])#,'ADF'])
                stats = dict(var=metrics,val=accur)
                sourceTable.data = stats
            
            except: 
                sourcePredLTG.data={'x': dt, 'y': empty}
                sourceFairPE.data={'x': dt, 'y': empty}
                sourceFairPX.data={'x': dt, 'y': empty}
                sourceTable.data = dict(var=[],val=[]) 
            
    refresh.on_click(callback)
    
    layout = column(row(menu, textbox1, refresh),
                    row(check, textbox2,),
                    row(plot0, plot1, table),
                    row(plot2, plot3),
                    row(textbox3))
    
    doc.add_root(layout)
   
show(bokeh_plot)



Loading BokehJS ...